## Database adapters
- `psycopg2` for PostgreSQL databases: https://www.psycopg.org/docs/usage.html

- `mysql-connector-python` for MySQL databases: https://www.w3schools.com/python/python_mysql_getstarted.asp

- `sqlite3` for SQLite databases: https://datacarpentry.org/python-ecology-lesson/09-working-with-sql/index.html


### Installing `psycopg2` on TACC's JupyterHub
In order to be able to import and use the `psycopg2` package for connecting to a database with Python, each user first needs to install the package in their own user space.

Here are two options for the installation:

#### 1. From Terminal
Install `psycopg2-binary` from the Terminal before opening a Jupyter Notebook.

Choose New > Terminal within Jupyter. Use `pip install` to install the package for your user:

    pip install --user psycopg2-binary 

Once the install is complete, launch a Notebook. 

Otherwise, the user site directory `/home/jupyter/mydata/jupyter_packages/lib/python3.6/site-packages` won't exist yet, and won't be appended to the `sys.path` for Python to find the package (this happens even if the package is installed via the Notebook, and then an import using Python is attempted).

#### 2. From Notebook
Install `psycopg2-binary` directly from a Notebook. Within a Notebook use: 

    !pip install --user psycopg2-binary 

Then, restart the kernel. Choose the upper menu option Kernal > Restart. 

The restart is necessary so that Python appends the user site directory mentioned aboce to `sys.path`, and then package can be imported using Python. Hooray!

### Confirming the package will be found
Confirm that the user site directory (`/home/jupyter/mydata/jupyter_packages/lib/python3.6/site-packages`) is in `sys.path`:

In [ ]:
import sys

sys.path

## Connecting to a PostgreSQL database
Import useful packages (aka "libraries") first.

In [ ]:
import psycopg2
import pandas as pd

Try connecting to a database.

In [ ]:
try:
    connection = psycopg2.connect("dbname='' user='' host='' password=''")
    print("Success!")
except:
    print("I am unable to connect to the database.")

## Performing database operations directly

Open a cursor to perform database operations.

In [ ]:
cursor = connection.cursor()

Write an SQL query, execute, and fetch the results. In this case, we're seeing which tables are in the database.

In [ ]:
sql_query = '''
SELECT table_name
FROM information_schema.tables
WHERE table_schema='public';
'''

cursor.execute(sql_query)

cursor.fetchall()

`cursor.fetchall()` fetches all the rows of a query result. It returns all the rows as a list of tuples. An empty list is returned if there is no record to fetch.

`cursor.fetchmany(size)` returns the number of rows specified by size argument. When called repeatedly this method fetches the next set of rows of a query result and returns a list of tuples. If no more rows are available, it returns an empty list.

`cursor.fetchone()` method returns a single record or None if no more rows are available.

Write an SQL query, execute, and commit. In this example, we're creating a new table.

In [ ]:
sql_create_table = '''
    CREATE TABLE animal_words
    (id INT PRIMARY KEY NOT NULL,
    animal TEXT NOT NULL,
    letter_count REAL);
    '''
    
cursor.execute(sql_create_table)
connection.commit()

Insert data into a table safely by passing parameters to the SQL statement. The `cursor.execute` function will take two arguments, the SQL query and the data to insert.

More details here: https://www.psycopg.org/docs/usage.html#query-parameters

In [ ]:
sql_insert = "INSERT INTO animal_words (id, animal, letter_count) VALUES (%s, %s, %s);"
record = (1, 'rhino', 5)

cursor.execute(sql_insert, record)
connection.commit()

Check the table contents. In this case, we're writing the SQL query directly into the `cursor.execute` function as a string.

In [ ]:
cursor.execute("SELECT * FROM animal_words;")
result = cursor.fetchall()

We also assigned the output of `cursor.fetchall` to a variable called `result`. We can call this new variable to see the output.

In [ ]:
result

## Bringing data from the database into a `pandas` DataFrame
Write an SQL query to get data, and use `pandas` to run the query and pass the results into a DataFrame object. 

This way you can work with the data using Python.

In [ ]:
species_df = pd.read_sql_query(
    '''
    SELECT *
    FROM species
    ''',
    connection)

View the first 10 rows of the new `species_df` DataFrame object.

In [ ]:
species_df.head(10)

## Closing communication with the database

In [ ]:
if(connection):
    cursor.close()
    connection.close()
    print("The database connection is closed.")